Date : Aug 3, 2020

<h1>Prime location in Lucknow, India for Real Estate investments</h1>
<h7>Dheerendra Kumar Singh</h7><br/>

<h4>Table of Contents</h4>
<ol>
<li>Introduction</li>
<li>Business Problem</li>
<li>Data</li>
<li>Methodology</li>
<li>Analysis</li>
<li>Results and Discussion</li>
<li>Conclusion</li>
</ol>

<h4>Introduction:</h4> 
A Real Estate investor Mr. XYZ from Mumbai, India has decided to expand his business in other parts of country. His business is to Buy, Renovate and Sell Real Estate properties. He decided to expand his business in Lucknow, the capital of UP, India. Since this is a new location for him, he decided to take help from professional data scientist to explore the prime location for Real Estate Investment in Lucknow.<br/>
About Lucknow : <a hreff="https://en.wikipedia.org/wiki/Lucknow">https://en.wikipedia.org/wiki/Lucknow</a>

<h4>Business Problem:</h4>
The task is to locate the hotspot for Real Estate investment in Lucknow, India. 

<h4>Audience:</h4>
Mr. XYZ from Mumbai, India is stakeholder who want to expand his business to new place. Before investing, Mr. XYZ want to explore the neighborhood and would be sure that his investment is profitable. 

<h4>Data acquisition:</h4>
<ol>
<li>Postal code data with coordinates.
Source: IN.zip file from http://www.geonames.org/export/zip/<br/>
    IN.zip file consist of tab seperated text file i.e. IN.txt. The file consists of Country Code, Postal Code, Location Name, State Name, State Code, City Name, District Name and Coordinates</li>
<li>FourSquare API to explore the location</li>
<li>Folium API to visualize result on map.</li>
</ol>

In [111]:
#import library
import pandas as pd
import folium
import foursquare
import requests

In [31]:
#load postal code file
pcode = pd.read_csv('IN.txt',sep='\t',header=None)
pcode.columns = ['CountryCode', 'PIN', 'Place', 'StateName', 'StateCode','City','A', 'District','B','Latitude','Longitude','Accuracy']
pcode.head()

,CountryCode,PIN,Place,StateName,StateCode,City,A,District,B,Latitude,Longitude,Accuracy
0,IN,744101,Marine Jetty,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.6667,92.7500,3
1,IN,744101,Port Blair,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.6667,92.7500,4
2,IN,744101,N.S.Building,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.6667,92.7500,3
3,IN,744102,Haddo,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.6833,92.7167,4
4,IN,744102,Chatham,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.7000,92.6667,3


In [37]:
pcode_lko = pcode[pcode['City']=='Lucknow']
pcode_lko.head()

,CountryCode,PIN,Place,StateName,StateCode,City,A,District,B,Latitude,Longitude,Accuracy
0,IN,226001,Secretariat (Lucknow),Uttar Pradesh,36,Lucknow,NaN,Lucknow,NaN,24.7307,82.6196,1
1,IN,226001,Lucknow G.P.O.,Uttar Pradesh,36,Lucknow,NaN,Lucknow,NaN,24.7307,82.6196,1
2,IN,226001,H C Bench,Uttar Pradesh,36,Lucknow,NaN,Lucknow,NaN,24.7307,82.6196,1
3,IN,226001,Jawahar Bhawan,Uttar Pradesh,36,Lucknow,NaN,Lucknow,NaN,24.7307,82.6196,1
4,IN,226001,Darul Safa,Uttar Pradesh,36,Lucknow,NaN,Lucknow,NaN,24.7307,82.6196,1


In [48]:
pcode_lko1 = pcode_lko[['PIN','StateName','City','Latitude','Longitude','Accuracy']]
pcode_lko1.drop_duplicates(inplace = True)
pcode_lko1 = pcode_lko1.reset_index(drop=True)
pcode_lko1.head()

<ipython-input-48-d693baae41a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pcode_lko1.drop_duplicates(inplace = True)


,PIN,StateName,City,Latitude,Longitude,Accuracy
0,226001,Uttar Pradesh,Lucknow,24.7307,82.6196,1
1,226002,Uttar Pradesh,Lucknow,27.8057,79.0009,4
2,226002,Uttar Pradesh,Lucknow,27.2736,79.9523,3
3,226002,Uttar Pradesh,Lucknow,26.7415,80.9037,4
4,226003,Uttar Pradesh,Lucknow,25.2813,88.4000,3


In [52]:
lko_lat = pcode_lko1['Latitude'].mean()
lko_lon = pcode_lko1['Longitude'].mean()
print('Co-ordinates',lko_lat,lko_lon)

Co-ordinates 26.468002247191013 81.1317988764045


In [59]:
# create map of New York using latitude and longitude values
map_lko = folium.Map(location=[lko_lat, lko_lon], zoom_start=6)

# add markers to map
for lat, lng, borough, neighborhood in zip(pcode_lko1['Latitude'], pcode_lko1['Longitude'], pcode_lko1['City'], pcode_lko1['PIN']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lko)  
    
map_lko

In [113]:
lko_lat = 26.8393
lko_lon = 80.9231
map_lko = folium.Map(location=[lko_lat, lko_lon], zoom_start=12)

df = pd.DataFrame(columns=['Latitude','Longitude'])

temp_lko_lat = lko_lat - 0.075
temp_lko_lon = lko_lon - 0.075
for i in range(0,10,1):
    temp2_lko_lat = temp_lko_lat + (0.02*i)
    for k in range(0,10,1):
        temp2_lko_lon = temp_lko_lon + (0.02*k)
        df = df.append({'Latitude': temp2_lko_lat,'Longitude':temp2_lko_lon}, ignore_index=True)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Latitude'], df['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lko)  
    
map_lko

In [116]:
df = df.reset_index()
df['index']

0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Name: index, Length: 100, dtype: int64

In [109]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ
CLIENT_SECRET:CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM


Function that extracts the category of the venue.

In [101]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods in Lucknow

In [103]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [117]:
lko_venues = getNearbyVenues(names=df['index'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
lko_venues.head()

0


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ&client_secret=CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM&v=20180604&ll=26.764300000000002,80.8481&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026C5DAB6BB0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

Let's check the size of the resulting dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

And let's examine the new dataframe size.

In [ ]:
toronto_onehot.shape

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each catego

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

toronto_grouped.shape

In [ ]:
Let's confirm the new size

In [ ]:
toronto_grouped.shape

Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's put that into a pandas dataframe First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h4>Methodology:</h4>
Real Estate that are closer to the School, Hospital, Restaurant, Clubs, Gym, Airport, Bus transport, etc are sell at a higher price and having more demand than others. The majority of people want to live close to where they work, shop and go out to enjoy themselves. We will use FourSquare API to explore the venues for each neighborhood


<h4>Analysis</h4>

<h4>Results and Discussion</h4>

<h4>Conclusion</h4>